In [1]:
import arcpy, calendar, os, re
from arcpy import env
from arcpy.sa import *
from otherfunctions import folders_exist
from datetime import date, timedelta

In [2]:
# Paths to input datasets
root_folder = r"Z:\PhD_Datasets&Analysis\Info_Inputs"
tam_out_dir = r"Z:\PhD_Datasets&Analysis\Outputs\T&M_WBM"
tc_ds = root_folder + "\\TerraClimate"
out_geotiff = tc_ds + "\\GeoTIFF"

# Set arcpy environment variables
env.overwriteOutput = True
arcpy.CheckOutExtension("spatial")
# env.cellSize = "MINOF" # Avoided to prevent huge files
env.cellSize = out_geotiff + "\\ppt_2023_1.tif" # Use TerraClimate resolution as reference for cell size
env.workspace = r"Z:\PhD_Datasets&Analysis\_ProcessingCache"
env.outputCoordinateSystem = arcpy.SpatialReference("WGS 1984") # WGS 1984 (4326)

In [3]:
# Get the current environment's spatial reference
spatial_ref = env.outputCoordinateSystem

# Check if a spatial reference is set
if spatial_ref:
    print(f"Spatial Reference Name: {spatial_ref.name}")
    print(f"Spatial Reference WKID: {spatial_ref.factoryCode}")
else:
    print("No spatial reference is set in the current environment.")

Spatial Reference Name: GCS_WGS_1984
Spatial Reference WKID: 4326


In [4]:
# Read the recession constant (k) raster
k = Float(Raster(r"Z:\PhD_Datasets&Analysis\Inputs\monthly_k_recession_all.tif"))
one_minus_k = 1 - k

In [5]:
######################################################
### Starting values for the water balance model - T&M
######################################################

# TerraClimate available period
terra_st_yr = 1958
terra_ed_yr = 2023
months = range(1, 12 + 1)  # Months from January (1) to December (12)

# Base flow of the previous month (mm) (bf0)
bflow_ant_initial = 10

In [ ]:
# Generate a list of period_months in the format 'YYYY-MM'
period_months = [
    f"{year}-{month}"
    for year in range(terra_st_yr, terra_ed_yr + 1)
    for month in months
]

In [8]:
# Create folders for other variables of tam model
bflow_dir = tam_out_dir + '\\bflow2'
folders_exist([bflow_dir])

# Folder with percolation rasters resulting from the model
perc_dir = tam_out_dir + '\\perc'

In [ ]:
def process_months(period_months):
    # Find the last processed month by checking existing output files
    existing_files = os.listdir(bflow_dir)
    bflow_files = [f for f in existing_files if f.startswith("bflow_")]

    last_year = 0
    last_month = 0

    if bflow_files:
        # Extract year and month from filenames (format: bflow_YYYY_MM.tif)
        pattern = re.compile(r"bflow_(\d+)_(\d+)\.tif")
        for file in bflow_files:
            match = pattern.match(file)
            if match:
                year, month = int(match.group(1)), int(match.group(2))
                if year > last_year or (year == last_year and month > last_month):
                    last_year = year
                    last_month = month

        print(f"Found last processed month: {last_year}-{last_month}")

        # Find the next month to process
        if last_month == 12:
            next_year = last_year + 1
            next_month = 1
        else:
            next_year = last_year
            next_month = last_month + 1

        # Find the first month to process (format: YYYY-MM)
        first_month = f"{next_year}-{next_month}"

        # If that month isn't in our period_months, we're done
        if first_month not in period_months:
            print(f"Next month {first_month} not in period_months. All processing complete.")
            arcpy.CheckInExtension("spatial")
            arcpy.ClearEnvironment("workspace")
            print("\nDONE!!")
            return

        # Find where to start in the period_months list
        start_index = period_months.index(first_month)
        remaining_months = period_months[start_index:]

        # Find the most recent monthly file in the output directory
        last_month_path = os.path.join(bflow_dir, f"bflow_{last_year}_{last_month}.tif")
        if os.path.exists(last_month_path):
            bflow_ant = Raster(last_month_path)
            print(f"Resuming from {first_month} with bflow_ant from {last_month_path}")
        else:
            print(f"Warning: Could not find monthly file for {last_year}-{last_month}, using default value instead.")
            bflow_ant = bflow_ant_initial  # Reset to initial value if no monthly file found
    else:
        # No existing files, start from beginning
        remaining_months = period_months
        bflow_ant = bflow_ant_initial  # Initial value
        print("Starting fresh calculation.")

    # Process the remaining months
    for month_str in remaining_months:
        try:
            print("Processing month: " + month_str)
            year, month = map(int, month_str.split('-'))
            perc_month = Float(Raster(os.path.join(perc_dir, f"perc_{year}_{month}.tif")))
            bflow_month = (bflow_ant * k) + (perc_month * one_minus_k)

            output_path = os.path.join(bflow_dir, f"bflow_{year}_{month}.tif")
            bflow_month.save(output_path)
            print(f"\tSaving bflow raster: {output_path}\n")

            # bflow_ant is bflow_month for the next month
            bflow_ant = bflow_month

        except Exception as e:
            print(f"Error processing month {month_str}: {e}")
            print("You can restart the script to continue from this point.")
            break

    arcpy.CheckInExtension("spatial")
    arcpy.ClearEnvironment("workspace")
    print("\nDONE!!")

# Usage:
process_months(period_months)


Starting fresh calculation.
Processing month: 1958-01
	Saving bflow raster: Z:\PhD_Datasets&Analysis\Outputs\T&M_WBM\bflow2\bflow_1958_01.tif

Processing month: 1958-02
	Saving bflow raster: Z:\PhD_Datasets&Analysis\Outputs\T&M_WBM\bflow2\bflow_1958_02.tif

Processing month: 1958-03
